## Understanding Multithreading

### What is Multithreading?

**Multithreading** allows multiple threads (lightweight processes) to run concurrently within a single process. Threads share the same memory space but execute independently.

### Key Concepts:
- **Thread**: A separate flow of execution within a program
- **Shared Memory**: All threads can access the same variables and data
- **GIL (Global Interpreter Lock)**: Python's mechanism that allows only one thread to execute Python bytecode at a time
- **I/O Bound**: Tasks that spend time waiting for input/output operations

### Basic Threading Example

In [ ]:
import threading
import time

def simple_task(name, duration):
    """A simple task that simulates work"""
    print(f"Task {name} started")
    time.sleep(duration)  # Simulate work
    print(f"Task {name} completed")

# Sequential execution
print("=== Sequential Execution ===")
start_time = time.time()

simple_task("A", 2)
simple_task("B", 2) 
simple_task("C", 2)

sequential_time = time.time() - start_time
print(f"Sequential execution took: {sequential_time:.2f} seconds\n")

In [ ]:
# Multithreaded execution
print("=== Multithreaded Execution ===")
start_time = time.time()

# Create threads
threads = []
for name, duration in [("A", 2), ("B", 2), ("C", 2)]:
    thread = threading.Thread(target=simple_task, args=(name, duration))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

threaded_time = time.time() - start_time
print(f"Multithreaded execution took: {threaded_time:.2f} seconds")
print(f"Speed improvement: {sequential_time/threaded_time:.1f}x faster")

### Thread Synchronization Example

In [ ]:
import threading
import time

# Shared resource
counter = 0
lock = threading.Lock()

def increment_counter(name, iterations):
    """Increment shared counter safely"""
    global counter
    
    for i in range(iterations):
        # Acquire lock before modifying shared resource
        with lock:
            current_value = counter
            time.sleep(0.001)  # Simulate some processing
            counter = current_value + 1
        
        if (i + 1) % 100 == 0:
            print(f"Thread {name}: {i + 1} increments completed")

# Reset counter
counter = 0

print("=== Thread Synchronization Demo ===")
threads = []

# Create multiple threads incrementing the same counter
for i in range(3):
    thread = threading.Thread(target=increment_counter, args=(f"Thread-{i+1}", 300))
    threads.append(thread)
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

print(f"\nFinal counter value: {counter}")
print(f"Expected value: {3 * 300}")
print(f"Synchronization {'✅ SUCCESS' if counter == 900 else '❌ FAILED'}")

## Understanding Multiprocessing

### What is Multiprocessing?

**Multiprocessing** creates separate processes that run independently with their own memory space. Each process has its own Python interpreter and memory.

### Key Concepts:
- **Process**: Independent program execution with separate memory
- **No GIL**: Each process has its own Python interpreter
- **CPU Bound**: Tasks that require intensive computation
- **IPC (Inter-Process Communication)**: Methods to share data between processes

## 🎯 When to Use What?

### Choose **Multithreading** when:

✅ **I/O Bound Tasks**:
- File reading/writing
- Network requests (web scraping, API calls)
- Database operations
- User interface responsiveness

✅ **Shared State is Important**:
- Multiple threads need to access same data
- Memory usage needs to be minimal
- Fast context switching required

✅ **Examples**:
```python
# Web scraping multiple URLs
# Reading multiple files
# GUI applications
# Web servers handling requests
```

### Choose **Multiprocessing** when:

✅ **CPU Bound Tasks**:
- Mathematical calculations
- Image/video processing
- Data analysis and machine learning
- Cryptographic operations

✅ **True Parallelism Needed**:
- Multiple CPU cores available
- Tasks are independent
- Process isolation is important

✅ **Examples**:
```python
# Scientific computing
# Batch data processing
# Parallel algorithms
# Independent task processing
```

### Decision Matrix

| Your Task | Recommendation | Reason |
|-----------|----------------|--------|
| **Downloading 100 files** | 🧵 Threading | I/O bound, waiting for network |
| **Processing 100 images** | ⚡ Multiprocessing | CPU bound, can utilize multiple cores |
| **Web server handling requests** | 🧵 Threading | I/O bound, shared state |
| **Scientific simulations** | ⚡ Multiprocessing | CPU intensive, independent calculations |
| **Database queries** | 🧵 Threading | I/O bound, waiting for DB response |
| **Video encoding** | ⚡ Multiprocessing | CPU intensive, can parallelize |
| **Real-time chat app** | 🧵 Threading | I/O bound, need shared state |
| **Data analysis (pandas)** | ⚡ Multiprocessing | CPU bound, large datasets |

## 🚨 Common Pitfalls and How to Avoid Them

### Threading Pitfalls

In [ ]:
import threading
import time

# ❌ BAD: Race condition without synchronization
def demonstrate_race_condition():
    shared_counter = 0
    
    def increment_unsafe():
        nonlocal shared_counter
        for _ in range(100000):
            shared_counter += 1  # Not atomic!
    
    threads = []
    for _ in range(3):
        thread = threading.Thread(target=increment_unsafe)
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()
    
    print(f"❌ Unsafe result: {shared_counter} (expected: 300000)")

# ✅ GOOD: Proper synchronization
def demonstrate_thread_safety():
    shared_counter = 0
    lock = threading.Lock()
    
    def increment_safe():
        nonlocal shared_counter
        for _ in range(100000):
            with lock:
                shared_counter += 1  # Thread-safe!
    
    threads = []
    for _ in range(3):
        thread = threading.Thread(target=increment_safe)
        threads.append(thread)
        thread.start()
    
    for thread in threads:
        thread.join()
    
    print(f"✅ Safe result: {shared_counter} (expected: 300000)")

print("Threading Race Condition Demo:")
demonstrate_race_condition()
demonstrate_thread_safety()